In [1]:
import pandas as pd
import requests
import warnings
from dotenv import load_dotenv
import os
import json
from ipywidgets.widgets.widget_int import IntProgress

load_dotenv()
warnings.simplefilter("ignore", pd.errors.SettingWithCopyError)

In [2]:
deforestation = pd.read_csv("../datasets/checkpoints/forest_loss.csv")

display(deforestation.info())
deforestation

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187440 entries, 0 to 187439
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   country            187440 non-null  object 
 1   subnational1       187440 non-null  object 
 2   subnational2       187440 non-null  object 
 3   Lat                187440 non-null  float64
 4   Long               187440 non-null  float64
 5   elevation          187440 non-null  float64
 6   threshold          187440 non-null  int64  
 7   Year               187440 non-null  int64  
 8   gain_2000-2020_ha  187440 non-null  int64  
 9   extent_2010_ha     187440 non-null  int64  
 10  area_ha            187440 non-null  int64  
 11  extent_2000_ha     187440 non-null  int64  
 12  forest_loss_ha     187440 non-null  int64  
dtypes: float64(3), int64(7), object(3)
memory usage: 18.6+ MB


None

,country,subnational1,subnational2,Lat,Long,elevation,threshold,Year,gain_2000-2020_ha,extent_2010_ha,area_ha,extent_2000_ha,forest_loss_ha
0,Colombia,Amazonas,El Encanto,-1.747145,-73.209101,120.301346,0,2001,773,1027642,1027642,1027642,212
1,Colombia,Amazonas,El Encanto,-1.747145,-73.209101,120.301346,0,2002,773,1027642,1027642,1027642,182
2,Colombia,Amazonas,El Encanto,-1.747145,-73.209101,120.301346,0,2003,773,1027642,1027642,1027642,88
3,Colombia,Amazonas,El Encanto,-1.747145,-73.209101,120.301346,0,2004,773,1027642,1027642,1027642,169
4,Colombia,Amazonas,El Encanto,-1.747145,-73.209101,120.301346,0,2005,773,1027642,1027642,1027642,147
...,...,...,...,...,...,...,...,...,...,...,...,...,...
187435,Colombia,Vichada,Santa Rosalía,5.135381,-70.864961,116.421806,75,2018,6493,28283,432087,30129,23
187436,Colombia,Vichada,Santa Rosalía,5.135381,-70.864961,116.421806,75,2019,6493,28283,432087,30129,24
187437,Colombia,Vichada,Santa Rosalía,5.135381,-70.864961,116.421806,75,2020,6493,28283,432087,30129,38
187438,Colombia,Vichada,Santa Rosalía,5.135381,-70.864961,116.421806,75,2021,6493,28283,432087,30129,67


In [3]:
locations = deforestation[["Lat", "Long"]].drop_duplicates().reset_index(drop = True)
locations

,Lat,Long
0,-1.747145,-73.209101
1,-1.378972,-72.804280
2,-1.322004,-69.578386
3,-4.203165,-69.935907
4,-0.716580,-71.101650
...,...,...
1058,5.491592,-70.413995
1059,6.189912,-67.482570
1060,4.245270,-70.328300
1061,4.488860,-69.791680


In [4]:
try:
    rainfall = pd.read_csv("../datasets/checkpoints/rainfall.csv")
    rainfall["Date"] = pd.to_datetime(rainfall["Date"], format = "%Y-%m-%d")
except:
    rainfall = pd.DataFrame()
    
display(rainfall.info())
rainfall.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70533984 entries, 0 to 70533983
Data columns (total 5 columns):
 #   Column         Dtype         
---  ------         -----         
 0   Hour           int64         
 1   Precipitation  float64       
 2   Date           datetime64[ns]
 3   Lat            float64       
 4   Long           float64       
dtypes: datetime64[ns](1), float64(3), int64(1)
memory usage: 2.6 GB


None

,Hour,Precipitation,Date,Lat,Long
0,0,0.1,2010-01-01,-1.747145,-73.209101
1,1,0.1,2010-01-01,-1.747145,-73.209101
2,2,0.1,2010-01-01,-1.747145,-73.209101
3,3,0.1,2010-01-01,-1.747145,-73.209101
4,4,0.1,2010-01-01,-1.747145,-73.209101


In [5]:
done = rainfall.copy()
if len(done) > 0:
    done["Year"] = done["Date"].dt.year
    done = (
        done
            .drop(["Hour", "Precipitation", "Date"], axis = 1)
            .drop_duplicates()
            .reset_index(drop = True)
    )
    
done

,Lat,Long,Year
0,-1.747145,-73.209101,2010
1,-1.378972,-72.804280,2010
2,-1.322004,-69.578386,2010
3,-4.203165,-69.935907,2010
4,-0.716580,-71.101650,2010
...,...,...,...
8041,8.958192,-75.838165,2017
8042,9.237640,-75.813625,2017
8043,8.171571,-76.062313,2017
8044,8.200939,-76.224737,2017


In [6]:
min_year = 2010
max_year = 2022
total = len(locations) * (max_year - min_year + 1)
progress = IntProgress(0, 0, total, 1)
display(progress)

df = []
missing = []
print("Previously Found {} of {}".format(len(done), total))
try:
    for year in range(min_year, max_year + 1):
        min_date = pd.to_datetime("{}-01-01".format(year), format = "%Y-%m-%d").strftime("%Y-%m-%d")
        max_date = pd.to_datetime("{}-12-31".format(year), format = "%Y-%m-%d") .strftime("%Y-%m-%d")
        for i, row in locations.iterrows():
            existing = done.loc[
                (done["Lat"] == row["Lat"]) &
                (done["Long"] == row["Long"]) &
                (done["Year"] == year)
            ]
            if len(existing) > 0:
                progress.value += 1
                continue
            
            endpoint = "{}/timeline/{},{}/{}/{}".format(
                os.environ.get("WEATHER_URL"),
                row["Lat"],
                row["Long"],
                min_date,
                max_date
            )
            
            params = {
                "unitGroup": "metric",
                "key": os.environ.get("WEATHER_API_KEY")
            }
            
            response = requests.get(endpoint, params)
            if response.status_code == 429:
                raise Exception("Daily request limit reached. Try again tomorrow")
            
            try:
                result = json.loads(response.text)
                for day in result["days"]:
                    df.append(pd.DataFrame({
                        "Hour": list(range(24)),
                        "Precipitation": list(map(lambda x: x["precip"], day["hours"])),
                        "Date": day["datetime"],
                        "Lat": row["Lat"],
                        "Long": row["Long"]
                    }))
            except:
                missing.append(pd.DataFrame({
                    "Lat": [row["Lat"]],
                    "Long": [row["Long"]],
                    "Year": [year]
                }))
            
            progress.value += 1
except Exception as err:
    print(err)

print("Found {} of {}".format(progress.value, total))
print("Missing {} of {}".format(len(missing), total))

IntProgress(value=0, max=13819)

Previously Found 8046 of 13819
Daily request limit reached. Try again tomorrow
Found 9201 of 13819
Missing 0 of 13819


In [7]:
if len(missing) > 0:
    missing = pd.concat(missing)
else:
    missing = pd.DataFrame()
    
missing

""


In [8]:
if len(df) > 0:
    df = pd.concat(df).reset_index(drop = True)
    display(df.info())
    display(df.describe())
else:
    df = pd.DataFrame()
    
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10117800 entries, 0 to 10117799
Data columns (total 5 columns):
 #   Column         Dtype  
---  ------         -----  
 0   Hour           int64  
 1   Precipitation  float64
 2   Date           object 
 3   Lat            float64
 4   Long           float64
dtypes: float64(3), int64(1), object(1)
memory usage: 386.0+ MB


None

,Hour,Precipitation,Lat,Long
count,1.011780e+07,1.010183e+07,1.011780e+07,1.011780e+07
mean,1.150000e+01,3.131426e-01,5.520424e+00,-7.458473e+01
std,6.922187e+00,1.331373e+00,2.681142e+00,1.741909e+00
min,0.000000e+00,0.000000e+00,-4.203165e+00,-8.171853e+01
25%,5.750000e+00,0.000000e+00,3.870204e+00,-7.575012e+01
50%,1.150000e+01,0.000000e+00,5.411390e+00,-7.472248e+01
75%,1.725000e+01,2.000000e-01,6.924009e+00,-7.339794e+01
max,2.300000e+01,2.803000e+02,1.338142e+01,-6.704646e+01


,Hour,Precipitation,Date,Lat,Long
0,0,0.0,2017-01-01,3.526180,-67.412493
1,1,0.0,2017-01-01,3.526180,-67.412493
2,2,0.0,2017-01-01,3.526180,-67.412493
3,3,0.0,2017-01-01,3.526180,-67.412493
4,4,0.0,2017-01-01,3.526180,-67.412493
...,...,...,...,...,...
10117795,19,NaN,2018-12-31,2.892506,-72.133175
10117796,20,0.0,2018-12-31,2.892506,-72.133175
10117797,21,0.0,2018-12-31,2.892506,-72.133175
10117798,22,0.0,2018-12-31,2.892506,-72.133175


In [9]:
if len(df) > 0:
    rainfall["Date"] = rainfall["Date"].dt.strftime('%Y-%m-%d')
    rainfall_new = pd.concat([ rainfall, df ]).reset_index(drop = True)
    display(rainfall_new.info())
    display(rainfall_new.describe())
    display(rainfall_new)
    rainfall_new.to_csv("../datasets/checkpoints/rainfall.csv", quoting = False, index = False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80651784 entries, 0 to 80651783
Data columns (total 5 columns):
 #   Column         Dtype  
---  ------         -----  
 0   Hour           int64  
 1   Precipitation  float64
 2   Date           object 
 3   Lat            float64
 4   Long           float64
dtypes: float64(3), int64(1), object(1)
memory usage: 3.0+ GB


None

,Hour,Precipitation,Lat,Long
count,8.065178e+07,8.061156e+07,8.065178e+07,8.065178e+07
mean,1.150000e+01,2.602369e-01,5.569723e+00,-7.461878e+01
std,6.922187e+00,1.107324e+00,2.569344e+00,1.690394e+00
min,0.000000e+00,0.000000e+00,-4.203165e+00,-8.171853e+01
25%,5.750000e+00,0.000000e+00,4.263082e+00,-7.574878e+01
50%,1.150000e+01,0.000000e+00,5.516347e+00,-7.474094e+01
75%,1.725000e+01,2.000000e-01,6.849736e+00,-7.342148e+01
max,2.300000e+01,2.935000e+02,1.338142e+01,-6.704646e+01


,Hour,Precipitation,Date,Lat,Long
0,0,0.1,2010-01-01,-1.747145,-73.209101
1,1,0.1,2010-01-01,-1.747145,-73.209101
2,2,0.1,2010-01-01,-1.747145,-73.209101
3,3,0.1,2010-01-01,-1.747145,-73.209101
4,4,0.1,2010-01-01,-1.747145,-73.209101
...,...,...,...,...,...
80651779,19,NaN,2018-12-31,2.892506,-72.133175
80651780,20,0.0,2018-12-31,2.892506,-72.133175
80651781,21,0.0,2018-12-31,2.892506,-72.133175
80651782,22,0.0,2018-12-31,2.892506,-72.133175


In [10]:
rainfall_new["Year"] = rainfall_new["Date"].str.slice(0, 4)
rainfall_new["Precipitation"].isna().groupby(rainfall_new["Year"]).mean()

Year
2010    0.000000
2011    0.000000
2012    0.000000
2013    0.000000
2014    0.000000
2015    0.000250
2016    0.002102
2017    0.000444
2018    0.002313
Name: Precipitation, dtype: float64